<a href="https://colab.research.google.com/github/YusunPark/BigData/blob/main/%EC%A0%9C2%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 분석 연습하기

### 1.1 데이터 탐색하기

In [ ]:
# 패키지 불러오기
import pandas as pd

x_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/y_train.csv', encoding='CP949')

In [ ]:
# 데이터 형태 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(3500, 10)
(2482, 10)
(3500, 2)


In [ ]:
id = x_test['cust_id']

x_train = x_train.drop(columns = ['cust_id'])
x_test = x_test.drop(columns = ['cust_id'])
y_train = y_train.drop(columns = ['cust_id'])

In [ ]:
# 기초 동계향 확인 하기 -> 여기서 데이터 스케일링이 필요한지 파악
# 여기선 데이터 전처리가 필요하다 -> 스케일링!!
x_train.describe()

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


### 1.2 전처리하기


In [ ]:
# 불필요한 컬럼 삭제하기 -> 굳이?

# 결측치 처리하기 -> 결측치가 너무 많다.-> 0으로 처리
x_train.isnull().sum()

총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [ ]:
x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)
x_train.isnull().sum()

총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64

In [ ]:
# 범주형 변수 인코딩!! <- 분리해줘야함
### 파생변수 생성하기 (일단 해보고 성능이 너무 낮으면 해도 좋을 듯)
# 스케일링

# 데이터 상관관계분석 -> 너무 영향성 있는거 빼기
# 모델에 집어넣기
print(f'주 구매상품 개수 {x_train["주구매상품"].unique().size}')
print(f'주 구매지점 개수 {x_train["주구매지점"].unique().size}')

주 구매상품 개수 42
주 구매지점 개수 24


In [ ]:
# 라벨 인코딩으로 범주형 변수 변경
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encode_list = ['주구매상품', '주구매지점']

for x in encode_list:
  x_train[x] = encoder.fit_transform(x_train[x])
  x_test[x] = encoder.fit_transform(x_test[x])


In [ ]:
# 스케일링 하기

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)


In [ ]:
# 상관관계 확인하기 -> 이거도 굳이,,?

x_train.corr()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
총구매액,1.000000,0.700080,0.467686,-0.082916,-0.038724,0.659084,0.090022,0.014396,-0.212944
최대구매액,0.700080,1.000000,0.429504,-0.090729,-0.024819,0.374147,0.018980,0.022277,-0.115837
환불금액,0.467686,0.429504,1.000000,-0.056604,-0.045686,0.377570,-0.003871,-0.024707,-0.137362
주구매상품,-0.082916,-0.090729,-0.056604,1.000000,0.015874,-0.185275,-0.274178,-0.010018,0.032469
주구매지점,-0.038724,-0.024819,-0.045686,0.015874,1.000000,-0.059570,-0.080804,0.011990,0.035344
내점일수,0.659084,0.374147,0.377570,-0.185275,-0.059570,1.000000,0.225264,-0.010325,-0.293200
내점당구매건수,0.090022,0.018980,-0.003871,-0.274178,-0.080804,0.225264,1.000000,0.007659,-0.091151
주말방문비율,0.014396,0.022277,-0.024707,-0.010018,0.011990,-0.010325,0.007659,1.000000,0.003372
구매주기,-0.212944,-0.115837,-0.137362,0.032469,0.035344,-0.293200,-0.091151,0.003372,1.000000


In [ ]:
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size = 0.2)

In [ ]:
# 데이터 학습

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X_TRAIN, Y_TRAIN)


   0
0  1
1  0
2  0
3  0
4  0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_TEST, model.predict(X_TEST))

0.5395348837209302

In [ ]:

y_test_predicted = model.predict(x_test)

print(pd.DataFrame(y_test_predicted))

      0
0     1
1     0
2     0
3     0
4     0
...  ..
2477  1
2478  1
2479  0
2480  0
2481  0

[2482 rows x 1 columns]


In [ ]:
# 결과 예측하가
# model = DecisionTreeClassifier()
# model.fit(X_TRAIN, Y_TRAIN)
y_test_proba = model.predict_proba(x_test)
print(pd.DataFrame(y_test_proba))
print(pd.DataFrame(y_test_proba)[1])

        0    1
0     0.0  1.0
1     1.0  0.0
2     1.0  0.0
3     1.0  0.0
4     1.0  0.0
...   ...  ...
2477  0.0  1.0
2478  0.0  1.0
2479  1.0  0.0
2480  1.0  0.0
2481  1.0  0.0

[2482 rows x 2 columns]
0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2477    1.0
2478    1.0
2479    0.0
2480    0.0
2481    0.0
Name: 1, Length: 2482, dtype: float64


In [ ]:
result = pd.DataFrame(y_test_proba)[1]
final = pd.concat([id, result], axis=1).rename(columns={1:'gender'})


In [ ]:
final

,cust_id,gender
0,3500,1.0
1,3501,0.0
2,3502,0.0
3,3503,0.0
4,3504,0.0
...,...,...
2477,5977,1.0
2478,5978,1.0
2479,5979,0.0
2480,5980,0.0


In [30]:
# 코드 정리

# 라이브러리 / 파일 열기
import pandas as pd

x_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/y_train.csv', encoding='CP949')

# 테스트 데이터 cust_id저장 -> 결과 파일 생성때 쓰이기 위해서
id = x_test['cust_id']

# cust_id 칼럼 삭제하기
x_train.drop(columns=['cust_id'], inplace = True)
x_test.drop(columns=['cust_id'], inplace = True)
y_train.drop(columns=['cust_id'], inplace = True)

# 결측치 처리하기
x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)

# 라벨 인코딩 설정하기
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoding_list = ['주구매상품', '주구매지점']
for x in encoding_list:
  x_train[x] = encoder.fit_transform(x_train[x])
  x_test[x] = encoder.fit_transform(x_test[x])

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 학습용/검증용 데이터로 분리하기
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2)

# 모델 학습
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=10, criterion='entropy')
model.fit(X_TRAIN, Y_TRAIN)

y_test_proba = model.predict_proba(x_test)

# 성능확인하기
from sklearn.metrics import roc_auc_score

Y_TEST_PREDICTED = model.predict(X_TEST)
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

0.5703372159907966


## 분류모델

In [98]:
# 내가 해보는 decition tree

# 데이터 로딩
import pandas as pd

x_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/titanic_x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/titanic_x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/titanic_y_train.csv', encoding='CP949')

# test 데이터의 id값 따로 저장하기
id = x_test['PassengerId']
# x_train.drop(columns=['승객이름'], inplace = True)
# x_test.drop(columns=['승객이름'], inplace = True)

# id,승객 이름 삭제
x_train.drop(columns=['PassengerId', '승객이름'], inplace=True)
x_test.drop(columns=['PassengerId', '승객이름'], inplace=True)

# null값이 존재함. -> 나이, 객실번호, 선착장... 어케하지
# 나이.....? --> ???? 평균값으로 대체?
# 객실번호, 선착장은 삭제 죽은거랑 상관없음
x_train.drop(columns=['객실번호', '선착장'], inplace=True)
x_test.drop(columns=['객실번호', '선착장'], inplace=True)

mean_age = x_train['나이'].mean()
x_train.fillna(mean_age, inplace=True)

x_test.fillna(mean_age, inplace=True)

# 라벨 인코딩 -> 성별, 티켓번호
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encode_list = ['성별', '티켓번호']

for x in encode_list:
  x_train[x] = encoder.fit_transform(x_train[x])
  x_test[x] = encoder.fit_transform(x_test[x])

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 성능 평가를 위한 테스트 셋 분리
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델 학습
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_TRAIN, Y_TRAIN['Survived'])

Y_PREDICT = model.predict(X_TEST)

# 성능 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST['Survived'], Y_PREDICT))

# 제출 파일 생성

result_predict = pd.DataFrame(model.predict(x_test), columns=['Survived'])
print(result_predict)
final = pd.concat([id, result_predict], axis = 1)
print(final)

0.797835676867935
     Survived
0           0
1           0
2           0
3           0
4           1
..        ...
413         0
414         1
415         0
416         0
417         1

[418 rows x 1 columns]
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


In [93]:
# 책의 내용 따라하기 - xgboost사용

# 라이브러리 
import pandas as pd


x_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/titanic_x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/titanic_x_test.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/titanic_y_train.csv', encoding='CP949')

# test_id 를 따로 저장하기 / test_id 삭제하기
id = x_test['PassengerId']
x_train.drop(columns=['PassengerId'], inplace = True)
x_test.drop(columns=['PassengerId'], inplace = True)
# x_train.drop(columns=['PassengerId'], inplace = True)

# 티켓번호 값들은 중복이 전체 데이터의 50퍼가 넘는다 삭제
x_train.drop(columns=['티켓번호'], inplace=True)
x_test.drop(columns=['티켓번호'], inplace=True)

# 승객 이름 데이터도 필요없음
x_train.drop(columns=['승객이름'], inplace=True)
x_test.drop(columns=['승객이름'], inplace=True)

# 결측치 처리 -> 나이(0.08), 객실번호, 선착장
# 나이 -> 상관관계가 0.08로 낮아 삭제
# 객실번호 ->  결측치가 많고, 데이터가 다 다른 값으로 설정 따라서 삭제
# 선착장 -> 선착장은 생존과 관계가 있음-> 가장 많은 빈도를 가지는 s로 대체

x_train.drop(columns=['나이', '객실번호'], inplace=True)
x_test.drop(columns=['나이', '객실번호'], inplace=True)

x_train['선착장'].fillna('S', inplace=True)
x_test['선착장'].fillna('S', inplace=True)

# 범주형 인코딩 - 다른 방식으로 진행 
# 성별 - 수동 인코딩
# 선착장 - 원핫 인코딩

x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

dummy = pd.get_dummies(x_train['선착장'], drop_first = True).rename(columns={'Q':'선착장_Q', 'S':'선착장_S'})
x_train = pd.concat([x_train, dummy], axis=1)
x_train.drop(columns=['선착장'], inplace=True)

dummy = pd.get_dummies(x_test['선착장'], drop_first = True).rename(columns={'Q':'선착장_Q', 'S':'선착장_S'})
x_test = pd.concat([x_test, dummy], axis=1)
x_test.drop(columns=['선착장'], inplace=True)

# 성능평가를 위한 데이터 분리
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size = 0.2, random_state=10)

# xgboost 학습
from xgboost import XGBClassifier

# 모델 1 (하이퍼 파라미터 x)
# model = XGBClassifier(eval_metric = 'error')
# model.fit(X_TRAIN, Y_TRAIN['Survived'])

#모델 2 (하이퍼 파라미터 사용) - 큰 차이 없는 듯
model = XGBClassifier(n_estimators=100, max_depth=3, eval_metric='error')
model.fit(X_TRAIN, Y_TRAIN['Survived'])

# 성능 체크 
from sklearn.metrics import roc_auc_score

Y_PREDICT = model.predict(X_TEST)
print(roc_auc_score(Y_TEST['Survived'], Y_PREDICT))

# 결과 파일 제출
y_predict = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})
print(y_predict)

result = pd.concat([id, y_predict], axis = 1)
print(result)

result.to_csv('/content/sample_data/123.csv', index=False)


0.7836366142817756
     Survived
0           0
1           0
2           0
3           0
4           1
..        ...
413         0
414         1
415         0
416         0
417         0

[418 rows x 1 columns]
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


## 예측 모델

In [144]:
# 내가 한 모델 - XGBRegression 사용할 것

# 데이터 로딩
import pandas as pd

x_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/bike_x_train.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/bike_y_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/bike_x_test.csv', encoding='CP949')

# 결과 파일을 위한 인덱스 저장
index = x_test['datetime']

# x_train, y_train 합쳐서 data 만든 후에 상관분석 진행
data = pd.concat([x_train, y_train], axis=1)

# datetime 값 분리
x_train['datetime'] = pd.to_datetime(x_train['datetime'])
x_test['datetime'] = pd.to_datetime(x_test['datetime'])

list_ = [x_train, x_test]
for x in list_:
  x['year'] = x['datetime'].dt.year
  # x['month'] = x['datetime'].dt.month
  x['hour'] = x['datetime'].dt.hour
  # x['day'] = x['datetime'].dt.day
  # x['dayofweek'] = x['datetime'].dt.dayofweek

x_train.drop(columns=['datetime'], inplace=True)
x_test.drop(columns=['datetime'], inplace=True)

# 결측치 확인 및 제거 ->결측치 존재 x

# 범주형 변수 인코딩 - LabelEncoder -> 범주형 없음

# 데이터 스케일링 - StandardScaler -> 안한거랑 한거랑 비교해보기 ->데이터 범주 차이가 크지 않아서 동일함
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

# train, test 분리
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델 학습
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_TRAIN, Y_TRAIN['count'])

# 성능 평가
from sklearn.metrics import r2_score

Y_PREDICT = pd.DataFrame(model.predict(X_TEST)).rename(columns={0:'count'})
print(Y_PREDICT.min())
Y_PREDICT[Y_PREDICT['count'] <0] = 0
print(Y_PREDICT.min())
print(r2_score(Y_TEST['count'], Y_PREDICT))

# 결과 파일 생성
result = pd.concat([index, pd.DataFrame(model.predict(x_test))], axis=1).rename(columns = {0:'count'})
result

[07:59:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
count   -60.338997
dtype: float32
count    0.0
dtype: float32
0.9372554212131647


,datetime,count
0,2011-01-20 0:00,15.752739
1,2011-01-20 1:00,0.875600
2,2011-01-20 2:00,-3.611838
3,2011-01-20 3:00,-4.428042
4,2011-01-20 4:00,-4.428042
...,...,...
6488,2012-12-31 19:00,218.422287
6489,2012-12-31 20:00,149.157333
6490,2012-12-31 21:00,125.530457
6491,2012-12-31 22:00,89.636230


In [148]:
# 교재 내용

# 데이터 가져오기
import pandas as pd
x_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/bike_x_train.csv', encoding='CP949')
y_train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/bike_y_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/bike_x_test.csv', encoding='CP949')

# 전처리 하기
list_ = [x_train, x_test]
for x in list_:
  x['datetime'] = pd.to_datetime(x['datetime'])
  x['year'] = x['datetime'].dt.year
  x['hour'] = x['datetime'].dt.hour
  
x_train.drop(columns=['datetime'], inplace=True)
x_test.drop(columns=['datetime'], inplace=True)
# y_train.drop(columns=['datetime'], inplace=True)

# 데이터 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델 학습
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=200, max_depth=5, random_state=10)
model.fit(X_TRAIN, Y_TRAIN['count'])

y_predict = pd.DataFrame(model.predict(x_test)).rename(columns={0:'count'})
y_predict[y_predict['count'] < 0] = 0

Y_PREDICT = pd.DataFrame(model.predict(X_TEST)).rename(columns={0:'count'})
Y_PREDICT[Y_PREDICT['count'] < 0] = 0

# 모델 평가하기
from sklearn.metrics import r2_score
print(r2_score(Y_TEST['count'], Y_PREDICT))


[08:21:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9372554212131647


In [54]:
data = pd.concat([x_train, y_train], axis=1)
data.corr()

,PassengerId,티켓등급,나이,형제자매배우자수,부모자식수,운임요금,癤풮assengerId,Survived
PassengerId,1.000000,-0.035144,0.036847,-0.057527,-0.001652,0.012658,1.000000,-0.005007
티켓등급,-0.035144,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.035144,-0.338481
나이,0.036847,-0.369226,1.000000,-0.308247,-0.189119,0.096067,0.036847,-0.077221
형제자매배우자수,-0.057527,0.083081,-0.308247,1.000000,0.414838,0.159651,-0.057527,-0.035322
부모자식수,-0.001652,0.018443,-0.189119,0.414838,1.000000,0.216225,-0.001652,0.081629
운임요금,0.012658,-0.549500,0.096067,0.159651,0.216225,1.000000,0.012658,0.257307
癤풮assengerId,1.000000,-0.035144,0.036847,-0.057527,-0.001652,0.012658,1.000000,-0.005007
Survived,-0.005007,-0.338481,-0.077221,-0.035322,0.081629,0.257307,-0.005007,1.000000


In [50]:
print(x_train['성별'].unique())
print(x_train['성별'].unique().size)

# nan값이 있어도 개수로 카운팅이 된다. ->그래서 작업형 1번에서 개수의 오류가 발생 했던 것.
# 이거를 해결 하기 위해서는 아예 null제거하고 개수를 세거나,,? 다른 방식을 외우기(책에 나옴)
print(x_train['선착장'].unique())
print(x_train['선착장'].unique().size)

['male' 'female']
2
['S' 'C' 'Q' nan]
4
